### Importing Files ###

In [ ]:
import pandas as pd
import glob
import os

In [ ]:
csv_files = glob.glob("inputs/*.csv")

dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    df['ETF'] = os.path.splitext(os.path.basename(file))[0]
    dfs.append(df)

etfs_tall = pd.concat(dfs, ignore_index=True)
etfs_tall.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True)
etfs_tall

### Calculating Returns ###

In [ ]:
etfs_tall['ret'] = etfs_tall.groupby('ETF')['Adj Close'].pct_change()

In [ ]:
etfs_tall.drop(columns=['Adj Close'], inplace=True)
etfs_tall

### Converting DF to Wide ###

In [ ]:
etfs_wide = etfs_tall.pivot(index='Date', columns='ETF', values='ret')
etfs_wide

### Keeping Rows with all ETFs present ###

In [ ]:
etfs_full = etfs_wide.dropna()
etfs_full = etfs_full.reset_index(drop=True)
etfs_full

### Creating input df for Dashboard ###

In [ ]:
etfs_full.to_csv('outputs/ready_to_sample.csv', index=False)

### Expanding Data (80 years) ###

In [ ]:
wide_sim_life = etfs_full.sample(n=12*80, random_state=42, replace=True, ignore_index=True).reset_index()
wide_sim_life = wide_sim_life.rename(columns={'index': 'Month'})
wide_sim_life

### Creating Additional Portfolios ###

In [ ]:
wide_sim_life['SPY_VFWAX'] = 0.5*wide_sim_life['SPY'] + 0.5 * wide_sim_life['VFWAX'] #domestic/international stock split
wide_sim_life['SPY_BND'] = 0.5*wide_sim_life['SPY'] + 0.5 * wide_sim_life['BND'] #stock/bond split
wide_sim_life['SPY_VNQ_BND'] = 0.5*wide_sim_life['SPY'] + 0.5 * wide_sim_life['VNQ'] #stock/bond/real estate split
wide_sim_life['SPY_VFWAX_BND'] = 0.4*wide_sim_life['SPY'] + 0.4 * wide_sim_life['VFWAX'] + 0.2 * wide_sim_life['BND'] #domestic/ international stock/bond split

In [ ]:
wide_sim_life

In [ ]:
final_data = wide_sim_life.reset_index().melt(id_vars='Month', var_name='ETF', value_name='ret')
final_data = final_data[final_data['ETF'] != 'index']

In [ ]:
final_data

In [ ]:
final_data.to_csv('outputs/etf_returns.csv', index=False)